In [0]:
import numpy as np
if np.__version__ != '1.16.2':
  !pip install numpy==1.16.2

In [2]:
import os
if not 'CarND-Alexnet-Feature-Extraction/' in os.getcwd():
  !git clone https://github.com/alex4men/CarND-Alexnet-Feature-Extraction
  !cd CarND-Alexnet-Feature-Extraction/
  !wget http://video.udacity-data.com.s3.amazonaws.com/topher/2016/October/580a829f_train/train.p
  !wget http://video.udacity-data.com.s3.amazonaws.com/topher/2016/October/580d880c_bvlc-alexnet/bvlc-alexnet.npy
  os.chdir('CarND-Alexnet-Feature-Extraction/')

Cloning into 'CarND-Alexnet-Feature-Extraction'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 125 (delta 3), reused 4 (delta 1), pack-reused 115
Receiving objects: 100% (125/125), 298.33 KiB | 3.87 MiB/s, done.
Resolving deltas: 100% (70/70), done.
--2019-06-04 13:18:16--  http://video.udacity-data.com.s3.amazonaws.com/topher/2016/October/580a829f_train/train.p
Resolving video.udacity-data.com.s3.amazonaws.com (video.udacity-data.com.s3.amazonaws.com)... 52.217.0.51
Connecting to video.udacity-data.com.s3.amazonaws.com (video.udacity-data.com.s3.amazonaws.com)|52.217.0.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120646441 (115M) []
Saving to: ‘train.p’

train.p             100%[===================>] 115.06M  45.4MB/s    in 2.5s    

2019-06-04 13:18:19 (45.4 MB/s) - ‘train.p’ saved [120646441/120646441]

--2019-06-04 13:18:19--  http://video.udacity-dat

In [3]:
import pickle
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np

from alexnet import AlexNet

nb_classes = 43
epochs = 10
batch_size = 128

# Load traffic signs data.
with open('./train.p', 'rb') as f:
    data = pickle.load(f)

# Split data into training and validation sets.
X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], test_size=0.33, random_state=0)

# Define placeholders and resize operation.
features = tf.placeholder(tf.float32, (None, 32, 32, 3))
labels = tf.placeholder(tf.int64, None)
resized = tf.image.resize_images(features, (227, 227))

# Returns the second final layer of the AlexNet model,
# this allows us to redo the last layer for the traffic signs
# model.
# pass placeholder as first argument to `AlexNet`.
fc7 = AlexNet(resized, feature_extract=True)
# NOTE: `tf.stop_gradient` prevents the gradient from flowing backwards
# past this point, keeping the weights before and up to `fc7` frozen.
# This also makes training faster, less work to do!
fc7 = tf.stop_gradient(fc7)

FileNotFoundError: ignored

In [0]:
# Add the final layer for traffic sign classification.
shape = (fc7.get_shape().as_list()[-1], nb_classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.matmul(fc7, fc8W) + fc8b

# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.global_variables_initializer()

preds = tf.arg_max(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [4]:
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(nb_classes))
logits = tf.matmul(fc7, fc8W) + fc8b

# TODO: Define loss, training, accuracy operations.
# HINT: Look back at your traffic signs project solution, you may
# be able to reuse some the code.
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
loss_op = tf.reduce_mean(cross_entropy)
opt = tf.train.AdamOptimizer()
train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
init_op = tf.global_variables_initializer()

preds = tf.arg_max(logits, 1)
accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

NameError: ignored

In [0]:
# Train and evaluate the feature extraction model.
def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

In [0]:
with tf.Session() as sess:
    sess.run(init_op)

    for i in range(epochs):
        # training
        X_train, y_train = shuffle(X_train, y_train)
        t0 = time.time()
        for offset in range(0, X_train.shape[0], batch_size):
            end = offset + batch_size
            sess.run(train_op, feed_dict={features: X_train[offset:end], labels: y_train[offset:end]})

        val_loss, val_acc = eval_on_data(X_val, y_val, sess)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =", val_loss)
        print("Validation Accuracy =", val_acc)
        print("")